In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Hannanum
from gensim.models import Word2Vec
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


class GoogleSearchScraper:
    def __init__(self, chrome_options):
        self.chrome_options = chrome_options
    
    def get_first_search_result_url(self, query):
        # Chrome 옵션을 설정하여 WebDriver 초기화
        driver = webdriver.Chrome(options=self.chrome_options)
        driver.get(f"https://www.google.com/search?q={query}" + " 인재상")
        
        try:
            print("검색 결과 페이지에서 첫 번째 항목의 URL을 가져오는 중...")
            first_title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h3.LC20lb.MBeuO.DKV0Md"))
            )
            url = first_title.find_element(By.XPATH, "./parent::a").get_attribute("href")
            print("첫 번째 항목 URL을 성공적으로 가져왔습니다.")
        except Exception as e:
            print("Error occurred while getting the first search result URL:", e)
            url = None
        finally:
            driver.quit()
        
        return url


class TextAnalyzer:
    def __init__(self):
        self.hannanum = Hannanum()

    def extract_nouns_from_html(self, url):
        print("명사 추출 중...")
        print("사이트 주소:", url)  # 사이트 주소 출력
        response = requests.get(url, verify=False)
        if response.status_code != 200:
            print("HTML 가져오기에 실패했습니다. 상태 코드:", response.status_code)
            return []

        response.encoding = response.apparent_encoding
        html = response.text

        try:
            soup = BeautifulSoup(html, "html.parser")
        except Exception as e:
            print("HTML 파싱에 실패했습니다:", e)
            return []

        nouns = self.hannanum.nouns(html)
        print("명사 추출 완료.")
        return nouns



class KeywordExtractor:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model = self.load_word2vec_model()

    def load_word2vec_model(self):
        try:
            print("Word2Vec 모델을 로드하는 중...")
            model = Word2Vec.load(self.model_path)
            print("Word2Vec 모델을 성공적으로 로드했습니다.")
            return model
        except Exception as e:
            print("Error occurred while loading Word2Vec model:", e)
            return None

    def extract_related_keywords(self, nouns, target_words, threshold=0.6):
        related_keywords = set()
        for word in target_words:
            for noun in nouns:
                try:
                    similarity = self.model.wv.similarity(noun, word)
                    if similarity >= threshold:
                        related_keywords.add(noun)
                except KeyError:
                    pass
        return related_keywords


def main():
    query = input("검색어를 입력하세요: ")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36")
    chrome_options.add_argument("headless")

    scraper = GoogleSearchScraper(chrome_options)
    first_url = scraper.get_first_search_result_url(query)

    if first_url:
        analyzer = TextAnalyzer()
        nouns = analyzer.extract_nouns_from_html(first_url)
        print("추출된 명사의 갯수:", len(nouns))

        extractor = KeywordExtractor(model_path="wiki.model")
        if extractor.model:
            target_words = ["인재", "소통", "능력", "책임", "성실", "열정", "꿈", "사람"]
            related_keywords = extractor.extract_related_keywords(nouns, target_words)

            print("인재상에 해당하는 키워드:", len(related_keywords), "개")
            df = pd.DataFrame({"인재상에 해당하는 키워드(중복제거)": list(related_keywords)})
            print(df)
        else:
            print("Word2Vec 모델을 로드하지 못했습니다. 모델 경로를 확인하세요.")
    else:
        print("검색 결과 URL을 가져오지 못했습니다.")


if __name__ == "__main__":
    main()


The chromedriver version (122.0.6261.128) detected in PATH at c:\Users\kosmo\Desktop\kosmo_baeksn\project3\resumate\data\notebook\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.88); currently, chromedriver 123.0.6312.86 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry


검색 결과 페이지에서 첫 번째 항목의 URL을 가져오는 중...
첫 번째 항목 URL을 성공적으로 가져왔습니다.
명사 추출 중...
사이트 주소: https://www.keccorp.com/kr/company/working.asp


c:\Users\kosmo\Desktop\kosmo_baeksn\project3\resumate\data\.venv\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.keccorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


명사 추출 완료.
추출된 명사의 갯수: 985
Word2Vec 모델을 로드하는 중...
Word2Vec 모델을 성공적으로 로드했습니다.
인재상에 해당하는 키워드: 10 개
  인재상에 해당하는 키워드(중복제거)
0                  인재
1                  열정
2                  성과
3                  자기
4                  기술
5                  능력
6                  집념
7                  역량
8                  목표
9                  성실
